## Imports

In [1]:
import pandas as pd
import numpy as np
import re
import os
import folium
import geopandas as gpd

## Data Read-in

In [2]:
df = pd.read_csv('redfin_2023-11-01-12-01-10.csv')

In [3]:
gdf = gpd.read_file('Miami-Dade_Boundary.geojson')

## Data Clean

In [4]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [5]:
df = df.dropna(subset=['SOLD DATE'])

In [6]:
# Define list of desired months (excluding current month)
desired_months = ['October']

# Filter DataFrame to include only entries from desired months
df_filtered = df[df['SOLD DATE'].str.split('-', expand=True)[0].isin(desired_months)]

# Reset the index
df_filtered = df_filtered.reset_index(drop=True)

In [7]:
# Data checks
print(df_filtered['PRICE'].isna().value_counts())
print('-------')
print(df_filtered['$/SQUARE FEET'].isna().value_counts())
print('-------')
print(df_filtered['YEAR BUILT'].isna().value_counts())
print('-------')

PRICE
False    772
Name: count, dtype: int64
-------
$/SQUARE FEET
False    767
True       5
Name: count, dtype: int64
-------
YEAR BUILT
False    772
Name: count, dtype: int64
-------


In [8]:
sorted_df = df_filtered.sort_values(by='YEAR BUILT', ascending=False)
second_newest_building = sorted_df.iloc[2]
print(second_newest_building['URL'])

https://www.redfin.com/FL/Miami/2629-SW-31st-Ct-33133/home/42756397


In [9]:
df_filtered.loc[df_filtered['PRICE'] == '0']

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE


In [10]:
df_filtered['PRICE'] = pd.to_numeric(df_filtered['PRICE'])
df_filtered['$/SQUARE FEET'] = pd.to_numeric(df_filtered['$/SQUARE FEET'])
df_filtered['YEAR BUILT'] = pd.to_numeric(df_filtered['YEAR BUILT'])
df_filtered['LATITUDE'] = pd.to_numeric(df_filtered['LATITUDE'])
df_filtered['LONGITUDE'] = pd.to_numeric(df_filtered['LONGITUDE'])

In [11]:
df_filtered.sort_values(by='PRICE',ascending=True).head(20)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
31,PAST SALE,October-24-2023,Condo/Co-op,14930 SW 82nd Ter Unit 16-405,Miami,FL,33193.0,255.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/14930-SW-82nd-...,MARMLS,A11453381,N,Y,25.689294,-80.434976
17,PAST SALE,October-27-2023,Condo/Co-op,13480 NE 6th Ave #306,North Miami,FL,33161.0,67000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/North-Miami/13480-NE...,Beaches MLS,RX-10905392,N,Y,25.898940,-80.187168
70,PAST SALE,October-17-2023,Condo/Co-op,13480 NE 6th Ave #112,North Miami,FL,33161.0,79000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/North-Miami/13480-NE...,MARMLS,A11402639,N,Y,25.898940,-80.187168
687,PAST SALE,October-6-2023,Condo/Co-op,13480 NE 6th Ave #214,North Miami,FL,33161.0,95000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/North-Miami/13480-NE...,MARMLS,A11274026,N,Y,25.898940,-80.187168
335,PAST SALE,October-11-2023,Condo/Co-op,18707 NE 14th Ave #737,Miami,FL,33179.0,110000.0,1.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/18707-NE-14th-...,MARMLS,A11441371,N,Y,25.948034,-80.173170
249,PAST SALE,October-3-2023,Condo/Co-op,100 NE 203rd Ter #4,Miami Gardens,FL,33179.0,112000.0,1.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/100-NE-203rd-T...,MARMLS,A11430517,N,Y,25.961601,-80.200692
632,PAST SALE,October-10-2023,Condo/Co-op,20180 NE 2nd Ave Unit X9,Miami Gardens,FL,33179.0,115000.0,1.0,1.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/20180-NE-2nd-A...,MARMLS,A11387628,N,Y,25.958040,-80.198098
458,PAST SALE,October-4-2023,Condo/Co-op,445 NE 195th St #330,Miami,FL,33179.0,120000.0,1.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/445-NE-195th-S...,MARMLS,A11361883,N,Y,25.957092,-80.191390
419,PAST SALE,October-13-2023,Condo/Co-op,1770 NE 191st St Unit 115-1,Miami,FL,33179.0,130000.0,1.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1770-NE-191st-...,MARMLS,A11434370,N,Y,25.951003,-80.166485
633,PAST SALE,October-11-2023,Condo/Co-op,1780 NE 191st St Unit 205-2,Miami,FL,33179.0,134000.0,1.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Miami/1780-NE-191st-...,MARMLS,A11367292,N,Y,25.949923,-80.166133


In [12]:
print(df_filtered['URL'].iloc[31])

https://www.redfin.com/FL/Miami/14930-SW-82nd-Ter-33193/unit-16-405/home/43318900


In [13]:
# Correct the prices, if needed
df_filtered.at[31,'PRICE']=(225000)

In [14]:
# Find problem psf by searching for a '0' value
df_filtered.loc[df_filtered['$/SQUARE FEET'] == '0'][['SOLD DATE','ADDRESS','CITY','$/SQUARE FEET','PRICE']]

,SOLD DATE,ADDRESS,CITY,$/SQUARE FEET,PRICE


In [15]:
# # Corrections, if needed
df_filtered.at[31,'$/SQUARE FEET']=(225000/912)

In [16]:
# Find problem psf by searching for low values
df_filtered.sort_values(by='$/SQUARE FEET',ascending=True).head(20)[['PRICE','ADDRESS','CITY','$/SQUARE FEET']]

,PRICE,ADDRESS,CITY,$/SQUARE FEET
70,79000.0,13480 NE 6th Ave #112,North Miami,87.0
17,67000.0,13480 NE 6th Ave #306,North Miami,92.0
687,95000.0,13480 NE 6th Ave #214,North Miami,112.0
458,120000.0,445 NE 195th St #330,Miami,120.0
636,160000.0,1770 NE 191st St Unit 810-1,Miami,122.0
419,130000.0,1770 NE 191st St Unit 115-1,Miami,122.0
536,165000.0,1780 NE 191st St Unit 811-2,Miami,125.0
451,265000.0,5300 NW 87th Ave #113,Doral,128.0
633,134000.0,1780 NE 191st St Unit 205-2,Miami,129.0
335,110000.0,18707 NE 14th Ave #737,Miami,130.0


In [17]:
print(df_filtered.URL.iloc[370])

https://www.redfin.com/FL/Doral/8290-Lake-Dr-33166/unit-416/home/43360344


In [18]:
# # Drop sales that aren't condos but labeled as such
# df_filtered = df_filtered.drop(1320)

## Make Maps

In [19]:
### Create a price column formatted as currency ###
df_filtered['PRICE_AS_CURRENCY'] = df_filtered['PRICE'].apply(lambda x: "${:,.0f}".format(x))
### Set formatting for Beds, Baths ###
df_filtered['YEAR BUILT DISPLAY'] = df_filtered['YEAR BUILT'].apply(lambda x: '{:.0f}'.format(x))
df_filtered['PRICE_SQUARE_FEET_AS_CURRENCY'] = df_filtered['$/SQUARE FEET'].apply(lambda x: '${:,.0f}'.format(x))

In [20]:
df_filtered = df_filtered.sort_values(by=['PRICE'], ascending=False)
### Insert different colors for top 10 sales vs. the rest ###
df_filtered['COLOR'] = ''
### Create RANK column ###
df_filtered['RANK'] = 0
### Insert RANK values ###
df_filtered['RANK'] = range(1, len(df_filtered) + 1)
# use numpy to assign values to the 'COLOR' column
df_filtered['COLOR'] = np.where(df_filtered['RANK'] <= 10, 'orange', 'blue')

## HTML Popup Formatter

In [21]:
### Define list of columns to drop from DF ###
columns_drop = ['SALE TYPE','PROPERTY TYPE','STATE OR PROVINCE','ZIP OR POSTAL CODE','HOA/MONTH','STATUS','NEXT OPEN HOUSE START TIME','NEXT OPEN HOUSE END TIME','SOURCE','MLS#','FAVORITE','INTERESTED','SQUARE FEET','LOT SIZE']

In [22]:
### Drop the columns ###
df_filtered = df_filtered.drop(columns=columns_drop)

In [23]:
def popup_html(row):
    Price = row['PRICE_AS_CURRENCY']
    Address = row['ADDRESS']
    City = row['CITY']
    sold_date = row['SOLD DATE']
    beds = row['BEDS']
    baths = row['BATHS']
    psf = row['PRICE_SQUARE_FEET_AS_CURRENCY']
    year_built = row['YEAR BUILT DISPLAY']
    rank = row['RANK']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Price: </strong>{}'''.format(Price) + '''<br>
    <strong>Address: </strong>{}'''.format(Address) + '''<br>
    <strong>City: </strong>{}'''.format(City) + '''<br>
    <strong>Sold: </strong>{}'''.format(sold_date) + '''<br>
    <strong>Beds: </strong>{}'''.format(beds) + '''<br>
    <strong>Baths: </strong>{}'''.format(baths) + '''<br>
    <strong>Price per sf: </strong>{}'''.format(psf) + '''<br>
    <strong>Year Built: </strong>{}'''.format(year_built) + '''<br>
    <strong>Price Rank: </strong>{}'''.format(rank) + '''
    </html>
    '''
    return html

In [24]:
### Create map container ###
m = folium.Map(location=df_filtered[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=10,tiles=None)

### Create title ###
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f"October 2023 Condo Sales")

m.get_root().html.add_child(folium.Element(title_html))

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top 10 Sales')

for index, row in df_filtered.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)
        
folium.GeoJson(gdf,tooltip='Miami-Dade County',name='Miami-Dade County').add_to(m)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)

# Display map
m

In [25]:
m.save('index.html')

## Summary Info

In [26]:
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

Newest = '\033[1m' + 'Newest' + '\033[0m'
Oldest = '\033[1m' + 'Oldest' + '\033[0m'

In [27]:
df_filtered.columns

Index(['SOLD DATE', 'ADDRESS', 'CITY', 'PRICE', 'BEDS', 'BATHS', 'LOCATION',
       'YEAR BUILT', 'DAYS ON MARKET', '$/SQUARE FEET', 'URL', 'LATITUDE',
       'LONGITUDE', 'PRICE_AS_CURRENCY', 'YEAR BUILT DISPLAY',
       'PRICE_SQUARE_FEET_AS_CURRENCY', 'COLOR', 'RANK'],
      dtype='object')

In [28]:
df_filtered['FULL_ADDRESS'] = df_filtered['ADDRESS'] + ' ' + df_filtered['CITY']

In [46]:
print(df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['URL'])

https://www.redfin.com/FL/North-Miami/13480-NE-6th-Ave-33161/unit-112/home/43450546


In [30]:
print(f"{ME}{BR}{df_filtered.loc[df_filtered['PRICE'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['PRICE'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['PRICE'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['PRICE'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['PRICE'].idxmax()]['YEAR BUILT']:.0f}")
print(f"{LE}{BR}{df_filtered.loc[df_filtered['PRICE'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['PRICE'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['PRICE'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['PRICE'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['PRICE'].idxmin()]['YEAR BUILT']:.0f}")

print(f"{MAX_PSF}{BR}{df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmax()]['YEAR BUILT']:.0f}")
print(f"{MIN_PSF}{BR}{df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['$/SQUARE FEET'].idxmin()]['YEAR BUILT']:.0f}")

print(f"{Newest}{BR}{df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['LOCATION']}, {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmax()]['YEAR BUILT']:.0f}")
print(f"{Oldest}{BR}{df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['LOCATION']}, {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['FULL_ADDRESS']} | Price ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['PRICE']:,.0f} | ${df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df_filtered.loc[df_filtered['YEAR BUILT'].idxmin()]['YEAR BUILT']:.0f}")

Most Expensive
BAYVIEW FISHER ISL NO TWO, 5373 Fisher Island Dr #5373 Miami Beach | Price $14,200,000 | $2,453 psf | Year built: 1991
Least Expensive
Zurich Condo, 13480 NE 6th Ave #306 North Miami | Price $67,000 | $92 psf | Year built: 1974
Highest Price Per Square Foot
CONTINUUM ON SOUTH BEACH, 50 S Pointe Dr #2501 Miami Beach | Price $10,000,000 | $3,482 psf | Year built: 2008
Lowest Price Per Square Foot
ZURICH CONDO, 13480 NE 6th Ave #112 North Miami | Price $79,000 | $87 psf | Year built: 1974
Newest
SOUTH BAY ESTATES, 2629 SW 31st Ct Miami | Price $1,360,000 | $575 psf | Year built: 2023
Oldest
THE SHERIDAN CONDO, 737 Jefferson Ave #201 Miami Beach | Price $525,000 | $445 psf | Year built: 1925


## Time on Market Calculator

In [31]:
# print(df_Current.loc[df_Current['YEAR BUILT'].idxmin()])

In [39]:
from datetime import datetime, timedelta

date1 = datetime(2023, 7, 19) ## List (Earlier) date
date2 = datetime(2023, 10, 27) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

100


## Map URL Snagger

In [33]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [34]:
cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/MD_condo_sales_month_ending_oct_2023


## Get Summary Data

In [35]:
print('SALES INFO')
print(f'Number of sales: {len(df_filtered)}')
print('--------')
print(f'Total sale price: ${df_filtered["PRICE"].sum():,.0f}')
print('--------')
print(f'Median sale price: ${df_filtered["PRICE"].median():,.0f}')
print('--------')
print(f'Max sale price: ${df_filtered["PRICE"].max():,.0f}')
print('--------')
print(f'Min sale price: ${df_filtered["PRICE"].min():,.0f}')
print('------------------------------------------------')
print('PSF INFO')
print(f'Max price per square foot: ${df_filtered["$/SQUARE FEET"].max():,.0f}')
print('--------')
print(f'Min price per square foot: ${df_filtered["$/SQUARE FEET"].min():,.0f}')
print('--------')
print(f'Median price per square foot: ${df_filtered["$/SQUARE FEET"].median():,.0f}')
print('------------------------------------------------')
print('CONDO AGES')
print(f'Newest building: {df_filtered["YEAR BUILT"].max()}')
print('----------')
print(f'Oldest building: {df_filtered["YEAR BUILT"].min()}')
print('----------')
print(f'Average building age: {df_filtered["YEAR BUILT"].mean()}')

SALES INFO
Number of sales: 772
--------
Total sale price: $481,399,583
--------
Median sale price: $386,250
--------
Max sale price: $14,200,000
--------
Min sale price: $67,000
------------------------------------------------
PSF INFO
Max price per square foot: $3,482
--------
Min price per square foot: $87
--------
Median price per square foot: $391
------------------------------------------------
CONDO AGES
Newest building: 2023.0
----------
Oldest building: 1925.0
----------
Average building age: 1988.2137305699482
